In [15]:
import tkcalendar
import tkinter as tk
import pandas as pd
import numpy as np
import requests
import matplotlib
import statsmodels
import sklearn
import flask
import yfinance as yf
import scipy
import tensorflow as tf
import tcn
import xgboost as xgb
import lightgbm as lgb
import prophet
import nest_asyncio

# Checking versions of the imported libraries
print(f"tkcalendar version: {tkcalendar.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"requests version: {requests.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")
print(f"statsmodels version: {statsmodels.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"flask version: {flask.__version__}")
print(f"yfinance version: {yf.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"tensorflow version: {tf.__version__}")
print(f"TCN version: {tcn.__version__}")
print(f"xgboost version: {xgb.__version__}")
print(f"lightgbm version: {lgb.__version__}")
print(f"prophet version: {prophet.__version__}")
try:
    print(f"nest_asyncio version: {nest_asyncio.__version__}")
except AttributeError:
    print("nest_asyncio does not have a __version__ attribute.")


tkcalendar version: 1.5.0
pandas version: 2.2.2
numpy version: 1.26.4
requests version: 2.32.2
matplotlib version: 3.8.4
statsmodels version: 0.14.2
scikit-learn version: 1.4.2
flask version: 3.0.3
yfinance version: 0.2.41
scipy version: 1.13.1
tensorflow version: 2.17.0
TCN version: 3.1.2
xgboost version: 2.1.1
lightgbm version: 4.5.0
prophet version: 1.1.5
nest_asyncio does not have a __version__ attribute.


C:\Users\yxu\AppData\Local\Temp\ipykernel_31076\2987977192.py:27: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(f"flask version: {flask.__version__}")


In [1]:
from tkcalendar import DateEntry
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import threading
import yfinance as yf
from scipy.stats import norm, multivariate_normal
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, GRU, MultiHeadAttention, Add, RepeatVector, GlobalAveragePooling1D, LayerNormalization, Embedding, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tcn import TCN, tcn_full_summary
import xgboost as xgb
import lightgbm as lgb
import warnings
from prophet import Prophet
import nest_asyncio
from threading import Thread
import time

## Web Application for Investment Simulation Using Historical Data

In [3]:
# Ensure the cache directory exists
CACHE_DIR = 'cache_data'
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)
# Read the data back from the TXT files
spy_data = pd.read_csv(os.path.join(CACHE_DIR, 'spy_data.txt'), sep='\t', parse_dates=['date'], index_col='date')
qqq_data = pd.read_csv(os.path.join(CACHE_DIR, 'qqq_data.txt'), sep='\t', parse_dates=['date'], index_col='date')
dia_data = pd.read_csv(os.path.join(CACHE_DIR, 'dia_data.txt'), sep='\t', parse_dates=['date'], index_col='date')
etf_data = {'SPY': spy_data, 'QQQ': qqq_data, 'DIA': dia_data}


In [7]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/simulate', methods=['POST'])
def simulate_investment():
    try:
        # Get form data
        symbol = request.form.get('symbol').upper()
        start_date = pd.to_datetime(request.form.get('start_date'))
        end_date = pd.to_datetime(request.form.get('end_date'))
        starting_principal = float(request.form.get('starting_principal'))
        auto_invest_amount = float(request.form.get('auto_invest_amount'))
        investment_day = request.form.get('investment_day')
        frequency = request.form.get('frequency')
        
        # Validate symbol
        if symbol not in etf_data:
            return "Invalid ETF symbol. Please select SPY, QQQ, or DIA."
        
        # Fetch ETF data
        df = etf_data[symbol]
        df = df[(df.index >= start_date) & (df.index <= end_date)]
        
        if df.empty:
            return "No data available for the selected date range."
        
        # Ensure date index is unique and sorted
        df = df.sort_index()
        df = df.loc[~df.index.duplicated(keep='first')]
        
        # Reset investment variables
        total_invested = starting_principal
        total_shares = starting_principal / float(df.iloc[0]['close'])
        current_balance = total_shares * float(df.iloc[0]['close'])
        
        # Track investment over time
        investment_values = [current_balance]
        total_investment_values = [total_invested]
        dates = [start_date]
        
        interval_days = {'weekly': 7, 'bi-weekly': 14, 'monthly': 30}
        
        date = start_date + timedelta(days=1)
        investment_interval = interval_days[frequency]
        last_investment_date = start_date
        while date <= end_date:
            if date in df.index:
                if (date - last_investment_date).days >= investment_interval and date.strftime('%A') == investment_day:
                    total_invested += auto_invest_amount
                    total_shares += auto_invest_amount / float(df.loc[date]['close'])
                    last_investment_date = date
                current_balance = total_shares * float(df.loc[date]['close'])
            
                # Record the values for plotting
                investment_values.append(current_balance)
                total_investment_values.append(total_invested)
                dates.append(date)
            
            date += timedelta(days=1)
        
        # Final calculation
        current_balance = total_shares * float(df.iloc[-1]['close'])
        profit = current_balance - total_invested
        monthly_savings = profit / ((end_date - start_date).days / 30)
        
        # Plot the results
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))
        ax1.plot(dates, investment_values, label='Investment Value Over Time')
        ax1.plot(dates, total_investment_values, label='Total Investment', linestyle='--')
        ax1.axhline(y=starting_principal, color='r', linestyle='--', label='Starting Principal')
        ax1.set_title(f'{symbol} Investment Simulation')
        ax1.set_ylabel('Investment Value (USD)')
        ax1.legend()

        profit_rates = [iv / ti for iv, ti in zip(investment_values, total_investment_values)]
        ax2.plot(dates, profit_rates, label='Profit Rate Over Time')
        ax2.set_title(f'{symbol} Profit Rate Simulation')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Profit Rate')
        ax2.legend()

        plt.tight_layout()

        # Save plot to a string buffer
        buf = io.BytesIO()
        plt.savefig(buf, format="png")
        buf.seek(0)
        plot_url = base64.b64encode(buf.getvalue()).decode('utf8')

        return render_template('result.html', plot_url=plot_url, starting_principal=starting_principal, 
                               total_invested=total_invested, total_shares=total_shares, 
                               current_balance=current_balance, profit=profit, 
                               monthly_savings=monthly_savings)
    except Exception as e:
        return str(e)

def run_flask():
    app.run(debug=False, use_reloader=False)

if 'flask_thread' in globals():
    flask_thread.join(0.1)

flask_thread = Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/simulate', methods=['POST'])
def simulate_investment():
    try:
        # Get form data
        symbol = request.form.get('symbol').upper()
        start_date = pd.to_datetime(request.form.get('start_date'))
        end_date = pd.to_datetime(request.form.get('end_date'))
        starting_principal = float(request.form.get('starting_principal'))
        auto_invest_amount = float(request.form.get('auto_invest_amount'))
        investment_day = request.form.get('investment_day')
        frequency = request.form.get('frequency')
        
        # Fetch ETF data (for simplicity, assuming it's a preloaded DataFrame 'etf_data')
        if symbol not in etf_data:
            return "Invalid ETF symbol. Please select SPY, QQQ, or DIA."
        
        df = etf_data[symbol]
        df = df[(df.index >= start_date) & (df.index <= end_date)]
        
        if df.empty:
            return "No data available for the selected date range."
        
        # Ensure date index is unique and sorted
        df = df.sort_index()
        df = df.loc[~df.index.duplicated(keep='first')]
        
        # Reset investment variables
        total_invested = starting_principal
        total_shares = starting_principal / float(df.iloc[0]['close'])
        current_balance = total_shares * float(df.iloc[0]['close'])
        
        # Track investment over time
        investment_values = [current_balance]
        total_investment_values = [total_invested]
        dates = [start_date]
        
        interval_days = {'weekly': 7, 'bi-weekly': 14, 'monthly': 30}
        date = start_date + timedelta(days=1)
        investment_interval = interval_days[frequency]
        last_investment_date = start_date

        while date <= end_date:
            if date in df.index:
                if (date - last_investment_date).days >= investment_interval and date.strftime('%A') == investment_day:
                    total_invested += auto_invest_amount
                    total_shares += auto_invest_amount / float(df.loc[date]['close'])
                    last_investment_date = date
                current_balance = total_shares * float(df.loc[date]['close'])
                investment_values.append(current_balance)
                total_investment_values.append(total_invested)
                dates.append(date)
            date += timedelta(days=1)
        
        # Final calculation
        current_balance = total_shares * float(df.iloc[-1]['close'])
        profit = current_balance - total_invested
        monthly_savings = profit / ((end_date - start_date).days / 30)
        
        # Plot the results
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))
        ax1.plot(dates, investment_values, label='Investment Value Over Time')
        ax1.plot(dates, total_investment_values, label='Total Investment', linestyle='--')
        ax1.axhline(y=starting_principal, color='r', linestyle='--', label='Starting Principal')
        ax1.set_title(f'{symbol} Investment Simulation')
        ax1.set_ylabel('Investment Value (USD)')
        ax1.legend()

        profit_rates = [iv / ti for iv, ti in zip(investment_values, total_investment_values)]
        ax2.plot(dates, profit_rates, label='Profit Rate Over Time')
        ax2.set_title(f'{symbol} Profit Rate Simulation')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Profit Rate')
        ax2.legend()

        plt.tight_layout()

        # Save plot to a string buffer
        buf = io.BytesIO()
        plt.savefig(buf, format="png")
        buf.seek(0)
        plot_url = base64.b64encode(buf.getvalue()).decode('utf8')

        return render_template('result.html', plot_url=plot_url, starting_principal=starting_principal, 
                               total_invested=total_invested, total_shares=total_shares, 
                               current_balance=current_balance, profit=profit, 
                               monthly_savings=monthly_savings)
    except Exception as e:
        return str(e)

# Run the Flask app directly in the notebook
app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
